<a href="https://colab.research.google.com/github/vijaygwu/algorithms/blob/main/253_Meeting_Rooms_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Given an array of meeting time intervals intervals where intervals[i] = [starti, endi], return the minimum number of conference rooms required.

**Example 1:**

Input: intervals = [[0,30],[5,10],[15,20]]
Output: 2

**Example 2:**

Input: intervals = [[7,10],[2,4]]
Output: 1

Constraints:

1 <= intervals.length <= 104
0 <= starti < endi <= 106



### 1. Quick problem recap  
Given a list of meeting **intervals** `[start, end]`, return the *minimum* number of conference rooms you’d need so that no two meetings in the same room overlap.

---

### 2. Key ideas the code relies on  

| Concept | Why it matters here |
|---------|--------------------|
| **Sort by start time** | Once meetings are in chronological order we can slide through them just once. |
| **Min‑heap (`heapq`) of end times** | The heap’s *root* is always the meeting that frees up first. That lets us instantly ask, “Is any room free yet?” in `O(log k)` time ( `k` = active rooms). |
| **Greedy reuse** | Whenever the earliest‑ending meeting finishes *before* the next meeting starts, we re‑use that room instead of opening a new one. |

---

### 3. Line‑by‑line commentary  

```python
class Solution:
    def minMeetingRooms(self, intervals: List[List[int]]) -> int:
```
`intervals` is `[[start, end], ...]` and we’ll return an `int`.

---

1. **Early exit**
   ```python
   if not intervals:
       return 0
   ```
   No meetings, no rooms—easy win.

2. **Prepare the heap**
   ```python
   free_rooms = []
   ```
   This will become a *min‑heap* of **end times** for currently occupied rooms.

3. **Sort intervals by when they start**
   ```python
   intervals.sort(key=lambda x: x[0])
   ```
   After this step we can iterate once, left→right.

4. **Give the first meeting a room**
   ```python
   heapq.heappush(free_rooms, intervals[0][1])
   ```
   ► `free_rooms = [end₀]` (heap with one item).

5. **Loop over the remaining meetings**
   ```python
   for start, end in intervals[1:]:
   ```
   For each upcoming meeting:

   **a. Is the earliest room free?**
   ```python
   if free_rooms[0] <= start:
       heapq.heappop(free_rooms)   # room released
   ```
   Because the heap’s root is the smallest end‑time, `free_rooms[0]` tells us the first room to free up.  
   *If* that end‑time ≤ this meeting’s **start**, that room is now empty → pop it off.

   **b. Book a room for this meeting**
   ```python
   heapq.heappush(free_rooms, end)
   ```
   Whether it’s a re‑used room or a brand‑new one, its new **end** time goes into the heap.

6. **Return the answer**
   ```python
   return len(free_rooms)
   ```
   By the time we’ve scheduled everything, the heap stores exactly one end‑time per room currently in use. Its length is therefore the minimum number of rooms ever needed.

---

### 4. Why the heap trick works  
Think of the heap as a *priority queue* ordered by “finishes soonest.”  
Whenever a meeting starts, we only care about *one* candidate room: the one due to free up first. If *that* room isn’t free yet, none of the others will be either (they end later). So checking the root alone is enough, and the heap keeps the root lookup at `O(1)` and push/pop at `O(log k)`.

---

### 5. Complexity  

| Step | Cost |
|------|------|
| Sort `n` intervals | `O(n log n)` |
| Each interval causes ≤ 1 push + ≤ 1 pop | `O(n log k)` but `k ≤ n`, so `O(n log n)` |

**Overall:** `O(n log n)` time, `O(k)` space (worst‑case `k = n` if nothing overlaps).  

That’s optimal for this approach—the sort cost dominates.

---

### 6. Mental picture (matching the live trace)

```
Sorted intervals:   [0,30]  [5,10]  [15,20]  [18,25]
Heap after each op: [30]
                    [10,30]
                    [30]           (room freed)
                    [20,30]
                    [20,30,25]
Answer ➜ 3 rooms
```

Room counts climb whenever we must push **without** a preceding pop. In the trace above they peak at 3, which is the minimum required.

---

**Tip:** If you ever forget why “heap of end times” works, imagine you’re Netflix’s conference‑room display. Each time a new meeting request pops up, you peek at the earliest‑ending room. If it’s still busy you light up *another* room; if it’s free you stick the new label on the same door. The maximum number of simultaneous sticky notes you ever have to post is the answer.

In [ ]:
class Solution:
    def minMeetingRooms(self, intervals: List[List[int]]) -> int:

        # If there is no meeting to schedule then no room needs to be allocated.
        if not intervals:
            return 0

        # The heap initialization
        free_rooms = []

        # Sort the meetings in increasing order of their start time.
        intervals.sort(key= lambda x: x[0])

        # Add the first meeting. We have to give a new room to the first meeting.
        heapq.heappush(free_rooms, intervals[0][1])

        # For all the remaining meeting rooms
        for i in intervals[1:]:

            # If the room due to free up the earliest is free, assign that room to this meeting.
            if free_rooms[0] <= i[0]:
                heapq.heappop(free_rooms)

            # If a new room is to be assigned, then also we add to the heap,
            # If an old room is allocated, then also we have to add to the heap with updated end time.
            heapq.heappush(free_rooms, i[1])

        # The size of the heap tells us the minimum rooms required for all the meetings.
        return len(free_rooms)